In [1]:
import importlib
from shapely.geometry import mapping
import models
import geopandas as gpd
importlib.reload(models)
from models import get_conn

engine, Session = get_conn()

In [ ]:
# Import Trails and Waypoints into Postgres. TODO: support the 4th dimension (time), which contains timestamps. Geopandas doesn't natively support it. also debug 'activities' which is a list of strings.

# change to import new data
def import_geojson_to_postgres(filename, notes=None):
    gdf = gpd.read_file(f'{filename}')
    columns = ['title','color','hexcolor','notes','track_type','routing_mode','distance','total_ascent','total_descent','stopped_time','total_time','average_speed','moving_time','moving_speed','latitude','longitude','geometry']
        
    gdf = gpd.GeoDataFrame(gdf[columns], geometry='geometry')
    if notes is not None:
        gdf['notes'] = notes
    trails = gdf[gdf['geometry'].geom_type == 'MultiLineString']
    points = gdf[gdf['geometry'].geom_type == 'Point'].drop(columns=['color', 'hexcolor', 'track_type', 'routing_mode', 'distance', 'total_ascent', 'total_descent', 'stopped_time', 'total_time', 'average_speed', 'moving_time', 'moving_speed'])

    with Session() as session:
        trails.to_postgis(name='trails', con=engine, if_exists='append')
        points.to_postgis(name='waypoints', con=engine, if_exists='append')

# filename = '/app/geojson_files/gaia_data.geojson'
# filename = '/app/geojson_files/international-travel.geojson'
# import_geojson_to_postgres(filename, "EDIT NOTE IF DESIRED")